<a href="https://colab.research.google.com/github/Z4HRA-S/NLP_Course_Spring2023/blob/main/Next_word_prediction_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/

1. Predicting Next Word
2. Pre-processing: Stem? 
3. Representing words, number or one-hots? 
4. LSTM, Motivation, Architecture, Concept

In [ ]:
# Small LSTM Network to Generate Text for Alice in Wonderland
import numpy as np
import nltk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer

In [ ]:
import string
from nltk.tokenize import word_tokenize
nltk.download("punkt")

def clean_text(text:str)->list:
    data = word_tokenize(text)
    data = [word.lower() for word in data]
    data = [d.translate(str.maketrans('', '', string.punctuation)) for d in data]
    data = [d for d in data if d.isalpha()]
    return data


def clean_text_char_level(data:str)->list:
    #data = word_tokenize(text)
    data = data.lower()
    data = data.translate(str.maketrans('', '', string.punctuation))
    return data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from nltk.corpus import brown
import nltk

nltk.download('brown')

data=brown.sents(categories=['news','reviews'])
data = " ".join(sum(data,[]))
data = clean_text_char_level(data)

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [ ]:
vocab = set(data)
print(len(vocab))

37


In [ ]:
"""from collections import Counter
vocab_count = dict(Counter(data))
low_freq_words = [k for k,v in vocab_count.items() if v<2]
print("words with one accourance in data: ", len(low_freq_words))"""

'from collections import Counter\nvocab_count = dict(Counter(data))\nlow_freq_words = [k for k,v in vocab_count.items() if v<2]\nprint("words with one accourance in data: ", len(low_freq_words))'

In [ ]:
"""from nltk.corpus import words
nltk.download("words")

valid_low_freq_words = list(filter(lambda x: x in words.words(), low_freq_words))"""

'from nltk.corpus import words\nnltk.download("words")\n\nvalid_low_freq_words = list(filter(lambda x: x in words.words(), low_freq_words))'

In [ ]:
"""len(valid_low_freq_words )"""

'len(valid_low_freq_words )'

In [ ]:
"""vocab=[k for k,v in vocab_count.items() if v>=3]
vocab.append("unk")
vocab=set(vocab)
#vocab.extend(valid_low_freq_words)
len(vocab)"""

'vocab=[k for k,v in vocab_count.items() if v>=3]\nvocab.append("unk")\nvocab=set(vocab)\n#vocab.extend(valid_low_freq_words)\nlen(vocab)'

In [ ]:
"""tokenizer = Tokenizer()
tokenizer.fit_on_texts(set(vocab))
sequences = tokenizer.texts_to_sequences(data)"""
word_to_id = {k:v for v,k in enumerate(vocab)}

In [ ]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 70
dataX = []
dataY = []
for i in range(0, len(data) - seq_length, 1):
	seq_in = data[i:i + seq_length]
	seq_out = data[i + seq_length]
	dataX.append([word_to_id[s] for s in seq_in])
	dataY.append(word_to_id[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  736304


In [ ]:
print(dataX[10])
print(dataY[10])

[29, 2, 7, 1, 23, 13, 9, 29, 35, 19, 22, 23, 33, 29, 6, 1, 19, 9, 29, 28, 22, 5, 33, 29, 4, 19, 5, 33, 22, 9, 29, 22, 23, 29, 5, 23, 27, 21, 28, 13, 5, 35, 22, 13, 5, 7, 23, 29, 7, 4, 29, 22, 13, 15, 22, 23, 13, 22, 28, 29, 19, 21, 2, 21, 23, 13, 29, 0, 19, 5]
3


In [ ]:
"""vocab_size = len(tokenizer.word_index) + 1"""
vocab_size = len(vocab)
print(vocab_size)

37


In [ ]:
dataX=np.array(dataX)/float(len(vocab))
dataY=np.array(dataY)
dataX=np.reshape(dataX, (dataX.shape[0], seq_length, 1))
dataY = to_categorical(dataY, num_classes=vocab_size)

print(dataX.shape, dataY.shape)

(736304, 70, 1) (736304, 37)


In [ ]:
from tensorflow.keras.layers import Flatten
import tensorflow as tf

model = Sequential()
model.add(LSTM(seq_length,return_sequences=True,input_shape=(dataX.shape[1],1)))
#model.add(Dropout(0.1))
model.add(LSTM(seq_length,return_sequences=True))
#model.add(Dropout(0.1))
model.add(LSTM(seq_length))
model.add(Dense(dataY.shape[1], activation='softmax'))
optimizer = tf.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 70, 70)            20160     
                                                                 
 lstm_7 (LSTM)               (None, 70, 70)            39480     
                                                                 
 lstm_8 (LSTM)               (None, 70)                39480     
                                                                 
 dense_2 (Dense)             (None, 37)                2627      
                                                                 
Total params: 101,747
Trainable params: 101,747
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(dataX, dataY, epochs=30, batch_size=128)

Epoch 1/30
5753/5753 [==============================] - 2171s 376ms/step - loss: 2.6456 - accuracy: 0.2422
Epoch 2/30
5753/5753 [==============================] - 2200s 382ms/step - loss: 2.3106 - accuracy: 0.3248
Epoch 3/30
5753/5753 [==============================] - 2150s 374ms/step - loss: 2.1861 - accuracy: 0.3588
Epoch 4/30
5753/5753 [==============================] - 2044s 355ms/step - loss: 2.1102 - accuracy: 0.3797
Epoch 5/30
1233/5753 [=====>........................] - ETA: 26:33 - loss: 2.0739 - accuracy: 0.3906

In [ ]:
text= "The 17th-century British empiricist John Locke held that linguistic meaning is mental: words are used to encode and convey thoughts, or ideas. Successful communication requires that the hearer correctly decode the speaker’s words into their associated ideas. So construed, the meaning of an expression, according to Locke, is the idea associated with it in the mind of anyone who knows and understands that expression."
text = clean_text_char_level(data)

model_input = [word_to_id[char] for char in text]

output_vector = [model.predict(model_input[i:i+10]) for i in range(len(text)-seq_length)]
idx = [np.argmax(v) for v in output_vector]